<a href="https://colab.research.google.com/github/GabiiKatherin/Analise_Corinthians_Brasileirao/blob/main/Analise_de_dados_Corinthians.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Instalando bibliotecas:

In [2]:
#instalando bibliotecas:
!pip install beautifulsoup4

import requests
#O BeautifulSoup é pra analisar e extrair dados do HTML ou XML
from bs4 import BeautifulSoup
import pandas as pd

2. Webscraping e consolidação dos dados:

In [14]:
#definindo as urls com os dados:
url_gols = 'https://www.espn.com.br/futebol/time/estatisticas/_/id/874/liga/BRA.1/vista'
url_cartoes = 'https://www.espn.com.br/futebol/time/estatisticas/_/id/874/liga/BRA.1/vista/cartoes'
url_rendimento = 'https://www.espn.com.br/futebol/time/estatisticas/_/id/874/liga/BRA.1/vista/rendimento'

In [4]:
#requisicao HTTP
response = requests.get(url_gols)
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
#procurando a tabela pela tag <table> - me certifiquei disso inspecionando o elemento na página
tables = soup.find_all('table')

In [6]:
#converte a tabela num Dataframe:
#a função read_html é do Pandas e basicamente converte o conteúdo HTML da tala num Dataframe.
df_list = []
for table in tables:
    df = pd.read_html(str(table))[0]
    df_list.append(df)

In [7]:
print(df_list[0])

IndexError: list index out of range

Essa etapa acusou um erro, dizendo que não havia nada na minha tabela, provavelmente porque os dados não foram capturados como eu gostaria. Então, fui testar e fazer uma exibição do que estava sendo capturado e identificar o problema.

In [8]:
#verificar o que foi capturado:
print(soup.prettify()[:1000])  #top 1000

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   ERROR: The request could not be satisfied
  </title>
 </head>
 <body>
  <h1>
   403 ERROR
  </h1>
  <h2>
   The request could not be satisfied.
  </h2>
  <hr noshade="" size="1px"/>
  Request blocked.
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
  <br clear="all"/>
  If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
  <br clear="all"/>
  <hr noshade="" size="1px"/>
  <pre>
Generated by cloudfront (CloudFront)
Request ID: BuYZyPR_CSEemxqUkgi30HT8MWosbV-BSxbw76LsQFMnCowZa2AgRQ==
</pre>
  <address>
  </address>
 </body>
</html>



Isso aqui é interessante! O que aconteceu foi que a minha requisição foi bloqueada pelo servidor. Provavelmente, porque ele está evitando acessos automatizados por scripts de scraping, uma medida de segurança super válida, inclusive.

Decidi testar mudar o User-Agent e em vez de usar o request padrão, imitar um navegador real.

In [24]:
#Define um cabeçalho HTTP para simular uma requisição feita por um navegador.
#O User-Agent especifica as informações do navegador e do sistema operacional
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
#Faz uma requisição GET na url que eu defini antes
response_g = requests.get(url_gols, headers=headers)
response_c = requests.get(url_cartoes, headers=headers)
response_r = requests.get(url_rendimento, headers=headers)

#Converte o HTML num soup - facilita a manipulação e extração dos dados.
soup_g = BeautifulSoup(response_g.text, 'html.parser')
soup_c = BeautifulSoup(response_c.text, 'html.parser')
soup_r = BeautifulSoup(response_r.text, 'html.parser')

In [30]:
#encontrando minha tabela:
tables = soup_g.find_all('table')
print(f"Total de tabelas encontradas: {len(tables)}")

Total de tabelas encontradas: 2


Como eu tenho mais de uma tabela, vou precisar de uma estrutura de repetição para capturar todas elas.

In [34]:
for t, table in enumerate(tables):
    df_gols = pd.read_html(str(table))[0]
    print(f"Tabela {t+1}:")
    print(df_gols)

Tabela 1:
     POS              Nome   J  G
0    1.0     Rodrigo Garro  25  5
1    2.0      Yuri Alberto  20  4
2    NaN      Ángel Romero  19  4
3    4.0              Cacá  21  3
4    5.0    Wesley Ribeiro  23  2
5    NaN      Talles Magno   6  2
6    7.0     Igor Coronado  22  1
7    NaN      Matheus Bidu  12  1
8    NaN           Giovane  11  1
9    NaN    Pedro Henrique  10  1
10  11.0           Raniele  23  0
11   NaN              Hugo  22  0
12   NaN        Mateuzinho  22  0
13   NaN       Breno Bidon  18  0
14   NaN      Félix Torres  17  0
15   NaN        Pedro Raúl  17  0
16   NaN            Fagner  12  0
17   NaN        Hugo Souza  11  0
18   NaN              Ryan  11  0
19   NaN  Gustavo Henrique  10  0
20   NaN     André Ramalho   9  0
21   NaN   Matheus Donelli   7  0
22   NaN     Carlos Miguel   6  0
23   NaN           Charles   6  0
Tabela 2:
     POS              Nome   J  A
0    1.0     Rodrigo Garro  25  4
1    NaN              Hugo  22  4
2    3.0      Yuri Alberto  

<ipython-input-34-a67a650f79a4>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_gols = pd.read_html(str(table))[0]
<ipython-input-34-a67a650f79a4>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_gols = pd.read_html(str(table))[0]


Subiu uma alerta de uso no pd.read_html(). E como minhas tabelas são pequenas, eu consegui notar que estão faltando dados. Por isso, vou utilizar o StringIO para tratar as strings como arquivo, mesmo.

In [36]:
tables = soup_c.find_all('table')

df_cartoes = pd.read_html(str(tables[0]))[0]
print("Tabela de Cartoes:")
print(df_cartoes)

Tabela de Cartoes:
     POS              Nome   J  CA  CV  pts
0    1.0  Gustavo Henrique  10   5   1    8
1    2.0           Raniele  23   6   0    6
2    NaN              Cacá  21   3   1    6
3    4.0      Alex Santana   8   4   0    4
4    NaN     Igor Coronado  22   4   0    4
5    NaN           Caetano   5   1   1    4
6    7.0           Charles   6   3   0    3
7    NaN      Félix Torres  17   3   0    3
8    NaN     André Ramalho   9   3   0    3
9   10.0              Hugo  22   2   0    2
10   NaN        Mateuzinho  22   2   0    2
11   NaN        Hugo Souza  11   2   0    2
12  13.0            Maycon   2   1   0    1
13   NaN          Léo Mana   5   1   0    1
14   NaN      Matheus Bidu  12   1   0    1
15   NaN            Fagner  12   1   0    1
16  17.0     Renato Santos   0   0   0    0
17   NaN    Diego Palacios   0   0   0    0
18   NaN      Felipe Longo   0   0   0    0
19   NaN   Matheus Donelli   7   0   0    0


<ipython-input-36-be9983ee573f>:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_cartoes = pd.read_html(str(tables[0]))[0]
